In [9]:
#importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
dataset = pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [11]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset=dataset.astype(int)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [12]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [13]:
#splitting training and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size=1/3, random_state=0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']}  
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [17]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [18]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [19]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'best'}: 0.874291385086922


In [20]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[74 11]
 [ 6 43]]


In [21]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.93      0.87      0.90        85
           1       0.80      0.88      0.83        49

    accuracy                           0.87       134
   macro avg       0.86      0.87      0.87       134
weighted avg       0.88      0.87      0.87       134



In [22]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8740696278511404

In [23]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.031514,0.041466,0.006983,0.002676,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",0.829615,0.850809,0.775815,0.870362,0.865054,0.838331,0.034293,5
1,0.005386,0.001018,0.003591,0.000798,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",0.804764,0.813511,0.740173,0.831098,0.811321,0.800173,0.031240,12
2,0.002794,0.000747,0.003591,0.000799,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.799620,0.868752,0.814409,0.924528,0.844370,0.850336,0.044141,3
3,0.002793,0.000747,0.004189,0.000977,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.885356,0.811321,0.777291,0.766556,0.850543,0.818213,0.044596,9
4,0.002793,0.000746,0.004388,0.000798,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.888107,0.832483,0.867097,0.885265,0.864018,0.021543,1
5,0.002394,0.000489,0.004189,0.000747,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.808927,0.825959,0.715207,0.867097,0.849057,0.813249,0.052866,10
6,0.004788,0.000747,0.004587,0.000798,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",0.822092,0.849057,0.832483,0.851527,0.923510,0.855734,0.035576,2
7,0.004987,0.001545,0.004787,0.000977,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",0.829615,0.850809,0.759721,0.773585,0.883278,0.819402,0.046544,8
8,0.002195,0.000398,0.003988,0.000631,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.849794,0.868752,0.759244,0.832483,0.883278,0.838710,0.043279,4
9,0.002792,0.001164,0.003789,0.000747,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.826263,0.811321,0.777291,0.831098,0.808775,0.810950,0.018859,11


In [24]:
age_input=float(input("Age:"))
EstimatedSalary_input=float(input("Estimated Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 32
Estimated Salary: 1222
Sex Male 0 or 1: 0


In [25]:
Future_Prediction=grid.predict([[age_input,EstimatedSalary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
